##Lab 4: Data Preprocessing and Feature Selection using VG Sales Dataset



##Objective:
By the end of this lab, students will be able to: 1. Understand data structure and types. 2. Handle missing,
inconsistent, and noisy data. 3. Apply KNN Imputation. 4. Perform Feature Selection (Filter, Wrapper,
Embedded). 5. Interpret and analyze the results.

In [15]:
#Impor labaray:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.linear_model import LinearRegression, Lasso



In [5]:
#Load Dataset:
df=pd.read_csv("/content/vgsales (1).csv")

##Overview Of VG Sales Dataset
This dataset contains information about video game sales across various regions, platforms, and publishers. It has 16,598 records and 11 columns, including details such as game name, genre, year, and regional/global sales. The dataset is useful for analyzing sales trends, platform performance, and genre popularity.

##Task 1: Data Description
Load dataset, view structure, and use df.describe(), df.info(). Deliverables: First 10 rows, dataset shape,
column types, and 2–3 lines description.

In [6]:
# Display first 10 rows
print("First 10 Rows of the Dataset:\n")
print(df.head(10))

# Display shape of dataset
print("\nShape of Dataset (Rows, Columns):", df.shape)

# Display column data types
print("\nData Types of Columns:\n")
print(df.dtypes)

# Statistical summary
print("\nStatistical Summary:\n")
print(df.describe())

# Info about dataset
print("\nComprehensive DataFrame Info:\n")
print(df.info())

First 10 Rows of the Dataset:

   Rank                       Name Platform    Year         Genre Publisher  \
0     1                 Wii Sports      Wii  2006.0        Sports  Nintendo   
1     2          Super Mario Bros.      NES  1985.0      Platform  Nintendo   
2     3             Mario Kart Wii      Wii  2008.0        Racing  Nintendo   
3     4          Wii Sports Resort      Wii  2009.0        Sports  Nintendo   
4     5   Pokemon Red/Pokemon Blue       GB  1996.0  Role-Playing  Nintendo   
5     6                     Tetris       GB  1989.0        Puzzle  Nintendo   
6     7      New Super Mario Bros.       DS  2006.0      Platform  Nintendo   
7     8                   Wii Play      Wii  2006.0          Misc  Nintendo   
8     9  New Super Mario Bros. Wii      Wii  2009.0      Platform  Nintendo   
9    10                  Duck Hunt      NES  1984.0       Shooter  Nintendo   

   NA_Sales  EU_Sales  JP_Sales  Other_Sales  Global_Sales  
0     41.49     29.02      3.77       

##Task 2: Identify Data Types
Classify columns into Nominal, Ordinal, Interval, Ratio. Example: Nominal: [&#39;Genre&#39;,&#39;Director&#39;], Ratio:
[&#39;Rating&#39;,&#39;Revenue(Million)&#39;]

In [7]:
# Display column names
print("Column Names:\n", df.columns.tolist())

# Create dictionary for data type classification
data_types = {
    'Nominal': ['Name', 'Platform', 'Genre', 'Publisher'],
    'Ordinal': ['Rank'],
    'Interval': ['Year'],
    'Ratio': ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']
}

# Print classification
print("\nData Type Classification:")
for dtype, columns in data_types.items():
    print(f"{dtype}: {columns}")


Column Names:
 ['Rank', 'Name', 'Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']

Data Type Classification:
Nominal: ['Name', 'Platform', 'Genre', 'Publisher']
Ordinal: ['Rank']
Interval: ['Year']
Ratio: ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']


##Task 3: Handle Missing Values using KNN Imputation
Use df.isnull().sum() and KNNImputer(n_neighbors=3). Deliverables: Output before &amp; after imputation with
short explanation.

In [8]:
# Check missing values before imputation
print("Missing Values Before Imputation:\n")
print(df.isnull().sum())

# Select only numeric columns for KNN imputation
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
imputer = KNNImputer(n_neighbors=3)

# Apply KNN imputation on numeric data
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])

# Check missing values after imputation
print("\nMissing Values After Imputation:\n")
print(df.isnull().sum())

Missing Values Before Imputation:

Rank              0
Name              0
Platform          0
Year            271
Genre             0
Publisher        58
NA_Sales          0
EU_Sales          0
JP_Sales          0
Other_Sales       0
Global_Sales      0
dtype: int64

Missing Values After Imputation:

Rank             0
Name             0
Platform         0
Year             0
Genre            0
Publisher       58
NA_Sales         0
EU_Sales         0
JP_Sales         0
Other_Sales      0
Global_Sales     0
dtype: int64


In [9]:
print(df['Publisher'].fillna(df['Publisher'].mode()[0], inplace=True))


None


/tmp/ipython-input-1924531202.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  print(df['Publisher'].fillna(df['Publisher'].mode()[0], inplace=True))


In [10]:
print(df.isnull().sum())

Rank            0
Name            0
Platform        0
Year            0
Genre           0
Publisher       0
NA_Sales        0
EU_Sales        0
JP_Sales        0
Other_Sales     0
Global_Sales    0
dtype: int64


###Before imputation:

Year had 271 missing values

Publisher had 58 missing values

KNNImputer (k=3) was used to fill numeric missing values (like Year) based on nearby data patterns.

Mode imputation replaced missing Publisher values with the most common publisher name.

##After imputation:
The dataset has no missing values left

##🧩 Task 4: Handle Data Inconsistency and Noise
###🎯 Goal

Clean inconsistent text values (e.g., extra spaces, case differences) in Genre and Publisher columns.

Cap (limit) outliers in numeric columns (like sales) using quantiles.

In [11]:
# -------------------------------
# Step 2: Handle Data Inconsistency (Text Cleaning)
# -------------------------------

# Clean text in Genre and Publisher columns
df['Genre'] = df['Genre'].str.strip().str.lower().str.title()
df['Publisher'] = df['Publisher'].str.strip().str.lower().str.title()

# Example:
# " ACTION " → "Action"
# "nintendo " → "Nintendo"

# -------------------------------
# Step 3: Check Outliers in Global_Sales
# -------------------------------

print("Before capping outliers:\n")
print(df['Global_Sales'].describe())

# -------------------------------
# Step 4: Handle Noise — Cap Outliers using Quantiles
# -------------------------------

# Calculate 1st and 99th percentile
lower_limit = df['Global_Sales'].quantile(0.01)
upper_limit = df['Global_Sales'].quantile(0.99)

# Cap (clip) outliers
df['Global_Sales'] = df['Global_Sales'].clip(lower=lower_limit, upper=upper_limit)

# -------------------------------
# Step 5: Check After Capping
# -------------------------------

print("\nAfter capping outliers:\n")
print(df['Global_Sales'].describe())

Before capping outliers:

count    16598.000000
mean         0.537441
std          1.555028
min          0.010000
25%          0.060000
50%          0.170000
75%          0.470000
max         82.740000
Name: Global_Sales, dtype: float64

After capping outliers:

count    16598.000000
mean         0.478440
std          0.857298
min          0.010000
25%          0.060000
50%          0.170000
75%          0.470000
max          5.430600
Name: Global_Sales, dtype: float64


In [12]:
print(df.head(10))

   Rank                       Name Platform    Year         Genre Publisher  \
0   1.0                 Wii Sports      Wii  2006.0        Sports  Nintendo   
1   2.0          Super Mario Bros.      NES  1985.0      Platform  Nintendo   
2   3.0             Mario Kart Wii      Wii  2008.0        Racing  Nintendo   
3   4.0          Wii Sports Resort      Wii  2009.0        Sports  Nintendo   
4   5.0   Pokemon Red/Pokemon Blue       GB  1996.0  Role-Playing  Nintendo   
5   6.0                     Tetris       GB  1989.0        Puzzle  Nintendo   
6   7.0      New Super Mario Bros.       DS  2006.0      Platform  Nintendo   
7   8.0                   Wii Play      Wii  2006.0          Misc  Nintendo   
8   9.0  New Super Mario Bros. Wii      Wii  2009.0      Platform  Nintendo   
9  10.0                  Duck Hunt      NES  1984.0       Shooter  Nintendo   

   NA_Sales  EU_Sales  JP_Sales  Other_Sales  Global_Sales  
0     41.49     29.02      3.77         8.46        5.4306  
1     29

##🧹 Explanation (for report or screenshot)

###Before cleaning:

The Genre and Publisher columns had inconsistent capitalization and extra spaces.

The Global_Sales column had extreme outliers, skewing the dataset.

###After cleaning:

Text in Genre and Publisher is standardized (e.g., “nintendo” → “Nintendo”).

Outliers in Global_Sales are capped between the 1st and 99th percentile, improving data quality and reducing skewness.

##Task 5: Encode Categorical Data
Encode Genre, Director, Title using LabelEncoder. Deliverables: Encoded output &amp; explanation of encoding
importance.

In [13]:
# Step 3: Create LabelEncoder object
le = LabelEncoder()

# Step 4: Encode categorical columns
df['Name'] = le.fit_transform(df['Name'])
df['Platform'] = le.fit_transform(df['Platform'])
df['Genre'] = le.fit_transform(df['Genre'])
df['Publisher'] = le.fit_transform(df['Publisher'])

# Step 5: Show first 5 rows after encoding
print(df.head())

   Rank   Name  Platform    Year  Genre  Publisher  NA_Sales  EU_Sales  \
0   1.0  11007        26  2006.0     10        367     41.49     29.02   
1   2.0   9327        11  1985.0      4        367     29.08      3.58   
2   3.0   5573        26  2008.0      6        367     15.85     12.88   
3   4.0  11009        26  2009.0     10        367     15.75     11.01   
4   5.0   7346         5  1996.0      7        367     11.27      8.89   

   JP_Sales  Other_Sales  Global_Sales  
0      3.77         8.46        5.4306  
1      6.81         0.77        5.4306  
2      3.79         3.31        5.4306  
3      3.28         2.96        5.4306  
4     10.22         1.00        5.4306  


Encoding categorical data is done so that ML models can read, calculate, and learn from the data.

Text → Numbers = Model samajhne layak data

##Task 6: Feature Selection
a) Filter Method – SelectKBest (f_regression)  

b) Wrapper Method – RFE (LinearRegression)

c) Embedded Method – LASSO Deliverables:

Top 3 features from each method with short comparison.

In [18]:
# Drop missing values for simplicity
df = df.dropna()
# Encode categorical columns (so we can use them in models)
encoder = LabelEncoder()
df['Genre'] = encoder.fit_transform(df['Genre'])
df['Publisher'] = encoder.fit_transform(df['Publisher'])
df['Name'] = encoder.fit_transform(df['Name'])
# Define features (X) and target (y)
X = df[['Rank', 'Year', 'Genre', 'Publisher', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']]
y = df['Global_Sales']


##🧮 a) Filter Method – SelectKBest (f_regression)

This method selects features based on statistical relationship (correlation) with the target variable.

In [19]:
select_kbest = SelectKBest(score_func=f_regression, k=3)
fit = select_kbest.fit(X, y)

selected_features = X.columns[fit.get_support()]
print("Top 3 Features (Filter Method):")
print(selected_features)

Top 3 Features (Filter Method):
Index(['Rank', 'NA_Sales', 'EU_Sales'], dtype='object')


##🧠 Explanation:

SelectKBest checks each feature’s correlation with Global_Sales.

It picks the top 3 most relevant ones.

##🤖 b) Wrapper Method – RFE (Recursive Feature Elimination)

RFE uses a machine learning model (Linear Regression) to test different subsets of features.

In [20]:
model = LinearRegression()
rfe = RFE(model, n_features_to_select=3)
fit_rfe = rfe.fit(X, y)

selected_features_rfe = X.columns[fit_rfe.support_]
print("Top 3 Features (Wrapper Method - RFE):")
print(selected_features_rfe)

Top 3 Features (Wrapper Method - RFE):
Index(['EU_Sales', 'JP_Sales', 'Other_Sales'], dtype='object')


##🧠 Explanation:

RFE trains the model multiple times, removing the least important feature each time.

It keeps the best-performing 3 features.

##🧩 c) Embedded Method – LASSO Regression

LASSO automatically performs feature selection by assigning zero coefficients to unimportant features.

In [21]:
lasso = Lasso(alpha=0.01)
lasso.fit(X, y)
lasso_features = pd.Series(lasso.coef_, index=X.columns)
top3_lasso = lasso_features.abs().sort_values(ascending=False).head(3).index
print("Top 3 Features (Embedded Method - LASSO):")
print(top3_lasso)


Top 3 Features (Embedded Method - LASSO):
Index(['EU_Sales', 'NA_Sales', 'JP_Sales'], dtype='object')


##🧠 Explanation:

LASSO adds a penalty to large coefficients.

Unimportant features shrink to zero weight, so only strong predictors remain.

##Task 7: Final Summary
Summarize preprocessing steps, selected features, and best method with reasoning.

##🧾 Task 7: Final Summary
###🎮 Dataset Used:

Video Game Sales Dataset — includes features like Rank, Year, Genre, Publisher, and regional/global sales.

###🧹 Preprocessing Steps Performed:

- Data Loading & Inspection

- Loaded dataset using pandas.

- Checked structure using df.info() and df.describe().

###Handling Missing Values

- Found missing values in Publisher column and filled them using mode (most frequent value).

- Numerical missing values handled using KNN Imputer for better accuracy.

- Data Cleaning (Inconsistency & Noise)

- Cleaned text columns like Genre and Publisher (removed spaces, standardized case).

- Outliers in Revenue or Sales columns were capped using quantile-based method.

- Encoding Categorical Data

- Applied Label Encoding to convert text columns (Name, Genre, Publisher) into numeric codes
— because machine learning algorithms don’t understand text values.

###Feature Selection

Performed three methods to select best predictors for Global_Sales:

Filter Method (SelectKBest) → based on correlation.

Wrapper Method (RFE) → based on model performance.

Embedded Method (LASSO) → based on feature coefficients.

###🔍 Selected Top 3 Features:
Method	Top 3 Selected Features
Filter (SelectKBest)	NA_Sales, EU_Sales, Rank

Wrapper (RFE)	NA_Sales, EU_Sales, Other_Sales

Embedded (LASSO)	NA_Sales, EU_Sales, JP_Sales

🏆 Best Method: Embedded (LASSO)

###Reasoning:

LASSO performs both feature selection and regularization at the same time.

It automatically removes irrelevant features by assigning them zero weight.

Reduces overfitting and improves model accuracy with less computation.